In [1]:
#-*- coding:utf-8
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""
stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde

import GameEnv
import random
import numpy as np
import DQN

gameEnv = GameEnv.GlodenFlower([20000,20000])
RLModel = DQN.DQN(embedding_size=50,sequence_length=20,learning_rate=0.01,batch_size=1000)
gameEnv.debug = False
memory = []
for episode in range(500000):
    # 初始化环境
    gameEnv.reset()

    playerI = gameEnv.getStartTurn()
    observation_this = [[],gameEnv.playerCards["A"],gameEnv.personStatus["A"]]
    if playerI == "B":
        availble_actions = gameEnv.chooseAvailbleAction(playerI)
        action,_ = RLModel.choose_action(np.array([observation_this]),availble_actions)
        #print (playerI, action, gameEnv.deskMoney, gameEnv.nowPrice)
        observation_next, reward, done = gameEnv.step(action, "B")
        playerI = "A"
        if done:
            continue

    while True:
        # DQN 根据观测值选择行为
        availble_actions = gameEnv.chooseAvailbleAction(playerI)
        action,_ = RLModel.choose_action(np.array([observation_this]),availble_actions)
        # 环境根据行为给出下一个 state, reward, 是否终止
        #print (playerI, action, gameEnv.deskMoney, gameEnv.nowPrice,gameEnv.personStatus[playerI])
        observation_next, reward, done = gameEnv.stepA(action,RLModel)


        # DQN 存储记忆
        # RL.store_transition(observation, action, reward)
        RLModel.store_transition(observation_this, action, reward,done,observation_next,gameEnv.playerCards["B"],gameEnv.personStatus["A"],gameEnv.nowPrice)

        # 控制学习起始时间和频率 (先累积一些记忆再开始学习)
        if (episode > 100000):
            if episode % 10000 == 0 :print "episode",episode
            if episode % 10 == 0:
                RLModel.train(RLModel.experience_replay())

        # 将下一个 state_ 变为 下次循环的 state
        observation_this = observation_next

        # 如果终止, 就跳出循环
        if done:
            break

        # end of game
print('game over')



Instructions for updating:
Use `argmax` instead
('loss', 100, 157.87308)
('loss', 200, 133.53827)
('loss', 300, 134.42966)
('loss', 400, 127.55225)
('loss', 500, 150.60524)
('loss', 600, 131.74232)
('loss', 700, 133.11278)
('loss', 800, 149.94525)
('loss', 900, 141.14423)
('loss', 1000, 148.1021)
('loss', 1100, 135.4074)
('loss', 1200, 135.74174)
('loss', 1300, 149.20432)
episode 110000
('loss', 1400, 152.97002)
('loss', 1500, 139.3742)
('loss', 1600, 143.33295)
('loss', 1700, 117.933876)
('loss', 1800, 152.78813)
('loss', 1900, 151.01576)
('loss', 2000, 107.953514)
('loss', 2100, 145.5142)
('loss', 2200, 158.56018)
('loss', 2300, 124.68183)
('loss', 2400, 136.21176)
('loss', 2500, 141.15924)
('loss', 2600, 160.41388)
('loss', 2700, 113.58865)
('loss', 2800, 131.70546)
('loss', 2900, 134.84297)
('loss', 3000, 141.78609)
episode 120000
episode 120000
('loss', 3100, 146.8801)
('loss', 3200, 114.44438)
('loss', 3300, 142.6611)


Exception: 没钱了4 Invalid level!

# 验证终结点是否有效

In [2]:
import pandas as pd
l = RLModel.card_index_dicts.keys()
l.remove(" ")
card_list = []
input_data = []
for i in range(100000):
    cards = random.sample(l,3)
    card_list.append(cards)
    input_data.append([[],cards ,"看"])

In [3]:
res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(np.array(input_data)))

In [4]:
df = pd.DataFrame(data={"card":card_list,"score":res[:,8]})

import GameEnv
gameEnv = GameEnv.GlodenFlower([2000,2000])
d = {"豹子":10,"同花顺":9,"金花":8,"顺子":7,"对子":6,"单":5}
reverse_dict = {v: k for k, v in d.iteritems()}

def f1(x):
    return reverse_dict[gameEnv.score(x)]
    
df['card_level'] = df['card'].apply(lambda x:f1(x))

In [5]:
df.groupby("card_level").count()

,card,score
card_level,,
单,74696,74696
同花顺,207,207
对子,16752,16752
豹子,245,245
金花,4993,4993
顺子,3107,3107


In [6]:
df.groupby("card_level").mean()

,score
card_level,
单,-6.210507
同花顺,11.997196
对子,6.602996
豹子,9.487987
金花,9.229124
顺子,8.725922


In [7]:
df.sort_values("score",ascending=False)[:]

,card,score,card_level
17738,"[spade_1, diamond_1, heart_1]",27.764511,豹子
98522,"[diamond_1, heart_1, spade_1]",27.764511,豹子
23076,"[diamond_1, spade_1, heart_1]",27.764511,豹子
23050,"[spade_1, diamond_1, heart_1]",27.764511,豹子
22926,"[spade_1, heart_1, diamond_1]",27.764511,豹子
7342,"[heart_1, spade_1, diamond_1]",27.764511,豹子
98505,"[heart_1, diamond_1, spade_K]",24.837688,对子
10187,"[heart_1, spade_K, diamond_1]",24.837688,对子
84626,"[diamond_1, spade_Q, heart_1]",23.911694,对子
71761,"[heart_1, spade_Q, diamond_1]",23.911694,对子


# 单点检查


In [10]:
# RLModel.train(RLModel.exerience_replay_final_step())
res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(np.array([[[],  ['club_10', 'heart_10', 'spade_10'],"看"]])))
for i,v in  zip(RLModel.action_reverse_index_dicts.keys(),res[0]):
    print i, RLModel.action_reverse_index_dicts[i],v

0 闷_2 0.0
1 闷_4 0.0
2 闷_8 0.0
3 闷开_0 0.0
4 看_2 9.598454
5 看_5 9.277475
6 看_10 11.547758
7 看_20 16.246895
8 开_0 8.205084
9 丢_0 -6.5036244


# 检查完整的一把过程

In [14]:
def print_prob(status):
    res = RLModel.sess.run(RLModel.predictions,feed_dict=RLModel._feed_dict(status))
    for i,v in  zip(RLModel.action_reverse_index_dicts.keys(),res[0]):
        print i, RLModel.action_reverse_index_dicts[i],v
import GameEnv
gameEnv = GameEnv.GlodenFlower([20000,20000])
gameEnv.reset()
gameEnv.playerCards["A"] = ['club_10', 'heart_10', 'spade_10']
gameEnv.playerCards["B"] = ['club_9', 'heart_9', 'spade_9']
print gameEnv.playerCards["A"],gameEnv.playerCards["B"],"\n"
playerI = "A"
while True:
        # DQN 根据观测值选择行为
        availble_actions = gameEnv.chooseAvailbleAction(playerI)
        action,action_value = RLModel.choose_action(np.array([observation_this]),availble_actions)
        print_prob(np.array([observation_this]))
        # 环境根据行为给出下一个 state, reward, 是否终止
        #print (playerI, action, gameEnv.deskMoney, gameEnv.nowPrice,gameEnv.personStatus[playerI])
        observation_next, reward, done = gameEnv.stepA(action,RLModel)

        # 将下一个 state_ 变为 下次循环的 state
        observation_this = observation_next
        print "______",playerI,action,action_value,"\n"
        # 如果终止, 就跳出循环
        if done:
            break
for i in observation_this[0]:
    print i

['club_10', 'heart_10', 'spade_10'] ['club_9', 'heart_9', 'spade_9'] 

0 闷_2 4.5685015
1 闷_4 5.9768696
2 闷_8 7.9051156
3 闷开_0 1.4748021
4 看_2 0.0
5 看_5 0.0
6 看_10 0.0
7 看_20 0.0
8 开_0 0.0
9 丢_0 -0.0
______ A 看_10 -1 

0 闷_2 0.0
1 闷_4 0.0
2 闷_8 0.0
3 闷开_0 0.0
4 看_2 12.273654
5 看_5 12.387595
6 看_10 15.152513
7 看_20 20.164433
8 开_0 9.47711
9 丢_0 -10.622791
______ A 看_20 20.164433 

0 闷_2 0.0
1 闷_4 0.0
2 闷_8 0.0
3 闷开_0 0.0
4 看_2 18.2842
5 看_5 18.925915
6 看_10 26.76909
7 看_20 38.419907
8 开_0 33.84288
9 丢_0 -33.943176
('\xe8\xb1\xb9\xe5\xad\x90', '\xe8\xb1\xb9\xe5\xad\x90', ['club_10', 'heart_10', 'spade_10'], ['club_9', 'heart_9', 'spade_9'])
______ A 开_0 -1 

A_看_10
B_看_10
A_看_20
B_看_20
A_开_0


In [23]:
_feed_dict = RLModel._feed_dict(np.array([observation_this]))
prob = RLModel.sess.run(RLModel.predictions, feed_dict=_feed_dict)[0]
max_value = -1000
max_action = "丢_0"
for action in availble_actions:
    if prob[RLModel.actions_index_dicts[action]] > max_value:
        print max_action,
        max_value = action

/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: unorderable dtypes; returning scalar but in the future this will be an error
  


In [25]:
print max_value

闷开_0


In [26]:
prob[RLModel.actions_index_dicts[action]]

12.817782

In [32]:
print RLModel.choose_action(np.array([observation_this]),availble_actions)

看_20


In [4]:
print action

('\xe7\x9c\x8b_20', -1)
